# Illustration of MUA signal

* First an extracellular spike is simulated with the model from Hay et al. (2011). The spike is calculated at many different locations
* We make a noisy signal, and insert the simulated EAPs in a "burst"
* The signal is high-pass filtered, rectified and then low-pass filtered

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import LFPy
import elephant
from brainsignals.plotting_convention import mark_subplots, simplify_axes
from brainsignals.neural_simulations import return_hay_cell
import brainsignals.neural_simulations as ns

ns.load_mechs_from_folder(ns.cell_models_folder)
np.random.seed(13734)

### First we simulate a spike with the Hay et al. (2011) model

In [ ]:
def insert_current_stimuli(cell):
    stim_params = {'amp': -0.4,
                   'idx': 0,
                   'pptype': "ISyn",
                   'dur': 1e9,
                   'delay': 0}
    
    synapse = LFPy.StimIntElectrode(cell, **stim_params)
    return synapse, cell

tstop = 150
dt = 2**-5

# Time window to extract spike from:
t0 = 115
t1 = 130

cell = return_hay_cell(tstop=tstop, dt=dt, make_passive=False)
ns.point_axon_down(cell)
syn, cell = insert_current_stimuli(cell)

cell.simulate(rec_imem=True, rec_vmem=True)
t0_idx = np.argmin(np.abs(cell.tvec - t0))
t1_idx = np.argmin(np.abs(cell.tvec - t1))

cell.vmem = cell.vmem[:, t0_idx:t1_idx]
cell.imem = cell.imem[:, t0_idx:t1_idx]
cell.tvec = cell.tvec[t0_idx:t1_idx] - cell.tvec[t0_idx]

xmin, xmax = [-70, 70]
ymin, ymax = [-70, 70]
zmin, zmax = [-70, 70]

dx = 10
x_grid, y_grid, z_grid = np.mgrid[xmin:xmax+dx:dx, ymin:ymax+dx:dx, zmin:zmax+dx:dx]

elec_grid_params = dict(
            sigma = 0.3,      # extracellular conductivity
            x = x_grid.flatten(),
            y = y_grid.flatten(),
            z = z_grid.flatten(),
            method = 'root_as_point',
        )

elec = LFPy.RecExtElectrode(cell, **elec_grid_params)
M_elec = elec.get_transformation_matrix()
eaps = M_elec @ cell.imem


## We create a noisy signal into which we insert the previously simulated spike

In [ ]:
filt_dict_mua_lp = {'highpass_freq': None,
             'lowpass_freq': 50,
             'order': 4,
             'filter_function': 'filtfilt',
             'fs': 1 / dt * 1000,
             'axis': -1}

filt_dict_high_pass = {'highpass_freq': 300,
             'lowpass_freq': None,
             'order': 4,
             'filter_function': 'filtfilt',
             'fs': 1 / dt * 1000,
             'axis': -1}


raw_sig_tstop = 500
num_spikes_to_insert = 1000
spike_burst_center = 250  # ms
burst_width = 50

# A cummulative sum of a random signal gives 1/f noise (strong noise at low frequencies)
num_tsteps = int(raw_sig_tstop / dt + 1)
tvec = np.arange(num_tsteps) * dt
sig_raw = np.zeros(num_tsteps)
sig_raw += np.cumsum(np.random.normal(0, 0.02, size=num_tsteps))

# Insert simulated spike into raw signal. All spikes are placed within a "burst"
num_spikes_on_grid = eaps.shape[0]
eap_len = eaps.shape[1]

insert_times = np.random.uniform(spike_burst_center - burst_width/2,
                                spike_burst_center + burst_width/2,
                                size=num_spikes_to_insert)

insert_time_idxs = np.array([np.argmin(np.abs(insert_time - tvec))
                        for insert_time in insert_times])

# The EAP was simulated at many locations and for each chosen time to insert a spike, 
# a random EAP is chosen.
chosen_eap_idxs = np.random.randint(0, num_spikes_on_grid,
                                            num_spikes_to_insert)
# Insert EAPs into raw signal
for s_idx in range(num_spikes_to_insert):
    t0_idx = insert_time_idxs[s_idx] - int(eap_len / 2)
    t1_idx = t0_idx + eap_len
    sig_raw[t0_idx:t1_idx] += eaps[chosen_eap_idxs[s_idx]]

# Filter signal
sig_hp = elephant.signal_processing.butter(sig_raw, **filt_dict_high_pass)
sig_hp_rect = np.abs(sig_hp) # Rectify
sig_hp_rect_lp = elephant.signal_processing.butter(sig_hp_rect,
                                                   **filt_dict_mua_lp)
    
    

In [ ]:
plt.close("all")
fig = plt.figure(figsize=[6, 1.8])
fig.subplots_adjust(left=0.07, bottom=0.18, top=0.65, right=0.98,
                    wspace=0.5)

ax_dict = dict(xlabel="time (ms)", ylabel="mV",
               xticks=[0, 250, 500], xlim=[-10, 510])

ax_spiketimes = fig.add_axes([0.07, 0.80, 0.17, 0.1], ylabel="#",
                             xticks=[0, 250, 500], xticklabels=[],
                             title="spike-time histogram",
                             xlim=[-10, 510])

ax_raw = fig.add_subplot(141, title="raw signal", **ax_dict)
ax_hp = fig.add_subplot(142, title="HP filtered", **ax_dict)
ax_hp_rect = fig.add_subplot(143, title="HP filtered\n+rectified",
                             **ax_dict)
ax_hp_rect_lp = fig.add_subplot(144,
                             title="HP filtered\n+rectified\n+LP filtered",
                             **ax_dict)

ax_spiketimes.hist(insert_times, facecolor='k')

ax_raw.plot(tvec, sig_raw, c='k')
ax_hp.plot(tvec, sig_hp, c='k')
ax_hp_rect.plot(tvec, sig_hp_rect, c='k')
ax_hp_rect_lp.plot(tvec[100:], sig_hp_rect_lp[100:], c='k')

for ax in [ax_raw, ax_hp, ax_hp_rect, ax_hp_rect_lp]:
    ax.axvspan(spike_burst_center - burst_width/2,
             spike_burst_center + burst_width/2, facecolor='0.8',)

mark_subplots(fig.axes[0], 'A', ypos=1.5, xpos=-0.2)
mark_subplots(fig.axes[1:], 'BCDE', xpos=-0.2)
simplify_axes(fig.axes)

fig.savefig("fig_hay_MUA.pdf")

